In [32]:
import __init__ # set up all the correct referenses
import os

subproject_name = "trajectories"

wd = "../../../"
project_path = os.path.realpath(wd)
print(project_path)

raw_data_dir = project_path+"/raw_data/"
print(raw_data_dir)


data_dir = project_path+f"/{subproject_name}"+"/data/"
print(data_dir)

results_dir = project_path+f"/{subproject_name}"+"/output"
print(results_dir)


/exports/reum/nsteinz/Project_trajectories
/exports/reum/nsteinz/Project_trajectories/raw_data/
/exports/reum/nsteinz/Project_trajectories/trajectories/data/
/exports/reum/nsteinz/Project_trajectories/trajectories/output


In [33]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt


In [34]:
das28 = [
    "pols links", "pols rechts", "pip 2 linkerhand", "pip 2 rechterhand", "pip 3 linkerhand", "pip 3 rechterhand",
    "pip 4 linkerhand", "pip 4 rechterhand",  "pip 5 linkerhand", "pip 5 rechterhand",
    "mcp 1 links", "mcp 1 rechts", "mcp 2 links", "mcp 2 rechts", "mcp 3 links", "mcp 3 rechts",
    "mcp 4 links", "mcp 4 rechts", "mcp 5 links", "mcp 5 rechts", "ip links", "ip rechts",
    "schouder links", "schouder rechts", 'elleboog links', 'elleboog rechts', 'knie links', 'knie rechts'
]
das44 = das28 + [
    'sternoclaviculair links', 'sternoclaviculair rechts', 'acromioclaviaculair rechts', 'acromioclaviculair links',
    "pip 2 linkervoet", "pip 2 rechtervoet", "pip 3 linkervoet", "pip 3 rechtervoet",
    "pip 4 linkervoet", "pip 4 rechtervoet",  "pip 5 linkervoet", "pip 5 rechtervoet",
    "bovenste spronggewricht links", "onderste spronggewricht links",
    "bovenste spronggewricht rechts","onderste spronggewricht rechts"
]    

In [35]:
name_map = {'IP links': 'ip links', 
 'IP rechts':'ip rechts',
 'IP voet links':'ip linkervoet',
 'IP voet rechts':'ip rechtervoet',
 'Manubrio sternaal gewricht':'manubrio sternaal gewricht',
 'acromioclaviaculair L':'acromioclaviculair links',
 'acromioclaviaculair R':'acromioclaviaculair rechts',
 'bovenste spronggewicht links':'bovenste spronggewricht links',
 'cmc 1 links':'cmc 1 links',
 'cmc 1 rechts':'cmc 1 rechts',
 'dip 2 links':'dip 2 linkerhand',
 'dip 2 links voet':'dip 2 linkervoet',
 'dip 2 rechts':'dip 2 rechterhand',
 'dip 2 rechts voet':'dip 2 rechtervoet',
 'dip 3 links':'dip 3 linkerhand',
 'dip 3 links voet':'dip 3 linkervoet',
 'dip 3 rechts':'dip 3 rechterhand',
 'dip 3 rechts voet':'dip 3 rechtervoet',
 'dip 4 links':'dip 4 linkerhand',
 'dip 4 links voet':'dip 4 linkervoet',
 'dip 4 rechts':'dip 4 rechterhand',
 'dip 4 rechts voet':'dip 4 rechtervoet',
 'dip 5 links':'dip 5 linkerhand',
 'dip 5 links voet':'dip 5 linkervoet',
 'dip 5 rechts':'dip 5 rechterhand',
 'dip 5 rechts voet':'dip 5 rechtervoet',
 'Elleboog L':'elleboog links',
 'elleboog R':'elleboog rechts',
 'pip 2 links hand':'pip 2 linkerhand',
 'pip 2 links voet':'pip 2 linkervoet',
 'pip 2 rechts hand':'pip 2 rechterhand',
 'pip 2 rechts voet':'pip 2 rechtervoet',
 'pip 3 links hand':'pip 3 linkerhand',
 'pip 3 links voet':'pip 3 linkervoet',
 'pip 3 rechts hand':'pip 3 rechterhand',
 'pip 3 rechts voet':'pip 3 rechtervoet',
 'pip 4 links hand':'pip 4 linkerhand',
 'pip 4 links voet': 'pip 4 linkervoet',
 'pip 4 rechts hand':'pip 4 rechterhand',
 'pip 4 rechts voet':'pip 4 rechtervoet',
 'pip 5 links hand':'pip 5 linkerhand',
 'pip 5 links voet':'pip 5 linkervoet',
 'pip 5 rechts hand':'pip 5 rechterhand',
 'pip 5 rechts voet':'pip 5 rechtervoet',
 'pols L':'pols links',
 'pols R':'pols rechts',
 'schouder L':'schouder links',
 'schouder R':'schouder rechts',
 'sternoclaviculair L':'sternoclaviculair links',
 'sternoclaviculair R':'sternoclaviculair rechts',
 'tarsometatarsaal L':'tarsometatarsaal Links',
 'tarsometatarsaal R':'tarsometatarsaal Rechts',
 'temporomandibulair L':'temporomandibulair links',
 'temporomandibulair R':'temporomandibulair rechts'}

In [36]:
def pivot_data(dataframe:pd.DataFrame, which: str, prefix:str="Pijn", JC_name:str="TJC" ) -> pd.DataFrame:
    """
    input:
    - dataframe
    - which:
    - prefix: waht is the prefix of the column names
    -JC-name: joint count name [TJC/SJC]
    
    """
    df = dataframe[["PATNR","DATUM","STELLING","XANTWOORD","ZANTWOORD","M_DATUM"]][dataframe["STELLING"] == which].copy()
    df["value"] = 1
    df = df.pivot_table(index=("PATNR","DATUM"), columns="XANTWOORD", values="value",fill_value=0)
    df = df.rename(columns=name_map)
    #print(df.columns)
    df["total "+which] = df.sum(axis=1)
    df[JC_name+"_28"] = df[das28].sum(axis=1)
    df[JC_name+"_44"] = df[das44].sum(axis=1)
    renaming = {x:prefix+"_"+"_".join(x.split(" ")) for x in df.columns[:-3]}
    df = df.rename(columns=renaming)
    return df.reset_index()

In [37]:
data = pd.read_csv(raw_data_dir+"data_03_12_22/"+"NewPatSelection_Mannequin_20221202.csv", sep="|", parse_dates=True,)


In [38]:
data = pd.read_csv(raw_data_dir+"data_03_12_22/"+"NewPatSelection_Mannequin_20221202.csv", sep="|", parse_dates=True,)
data["M_DATUM"] = pd.to_datetime(data.created.copy())
data['DATUM'] = pd.to_datetime(data.authored)
data["DATUM_use"] =data.DATUM.dt.date
data['DATUM']
data.columns


Index(['PATNR', 'PID', 'identifier_value', 'item_text',
       'item_answer1_value_original', 'item_answer2_value_original',
       'item_answer3_value_original', 'item_answer_valueCoding_display',
       'authored', 'questionnaire_Questionnaire_value', 'CATEGORIE',
       'author_Person_value', 'created', 'item_answer_lastUpdateDate', 'PEC',
       'M_DATUM', 'DATUM', 'DATUM_use'],
      dtype='object')

In [39]:
vas = data.item_text[data.item_text.str.contains("VAS")].unique()
vas_data = data.query("item_text in @vas")[["PATNR", "DATUM_use","item_text" ,"item_answer1_value_original"]]
vas_data.item_answer1_value_original = vas_data.item_answer1_value_original.astype(float )
vas_data = vas_data.pivot_table(index=("PATNR","DATUM_use"), columns="item_text", values="item_answer1_value_original",fill_value=np.NaN).reset_index()
vas_data.to_csv(data_dir+"vas.csv",index=False)

In [40]:
das_28 = data.query("item_text == 'Gezwollen gewrichten'")[["PATNR", "authored", "item_answer1_value_original"]]
das_28["DATUM"] = pd.to_datetime(das_28.authored).dt.date
das_28 = das_28.dropna(axis=0)
das_28["count"] = 1
#das_28["BSE_das"] = das_28.item_answer1_value_original.str.replace(",",".").str.strip('"').apply(lambda x: float(str(x).split("=")[-1]))


In [41]:
#das_28[["PATNR", "DATUM", "count"]].to_csv(data_dir+"bse_das.csv",index=False)

In [42]:
data = data.rename(columns={"item_text":"STELLING","item_answer_valueCoding_display":"XANTWOORD","item_answer3_value_original":"ZANTWOORD"})

In [43]:
ggdf = pivot_data(dataframe=data, which="Zwelling", prefix= "Gezwollen", JC_name="SJC")

In [44]:
pgdf = pivot_data(dataframe=data, which="Pijn", prefix= "Pijn", JC_name="TJC")

In [45]:
dataset = pgdf.merge(ggdf, how="outer", on=["PATNR","DATUM"]).fillna(0)

In [46]:
df = data[["PATNR","DATUM","STELLING","XANTWOORD","ZANTWOORD","M_DATUM"]][data["STELLING"] == "Geen pijnlijke gewrichten"].copy()
df["value"] = 1
test_pg = df.pivot_table(index=("PATNR","DATUM"), columns="STELLING", values="value",fill_value=0)
df = data[["PATNR","DATUM","STELLING","XANTWOORD","ZANTWOORD","M_DATUM"]][data["STELLING"] == "Geen gezwollen gewrichten"].copy()
df["value"] = 1
test_gg =df.pivot_table(index=("PATNR","DATUM"), columns="STELLING", values="value",fill_value=0)
test_merge = test_gg.merge(test_pg, how="outer",left_index=True, right_index=True).fillna(0)

In [47]:
dataset = dataset.merge(test_merge, how="outer", on=["PATNR","DATUM"]).fillna(0).drop(["Geen gezwollen gewrichten", "Geen pijnlijke gewrichten"],axis=1)

In [48]:
[["total Zwelling","SJC_28","SJC_44","total Pijn","TJC_28","TJC_44"]]
#dataset.drop(columns=["PATNR","DATUM","total Zwelling","SJC_28","SJC_44","total Pijn","TJC_28","TJC_44"]).to_csv("vae_training_data.csv", index = False)

[['total Zwelling', 'SJC_28', 'SJC_44', 'total Pijn', 'TJC_28', 'TJC_44']]

In [49]:
print(len(dataset.PATNR.unique()))
len(dataset)
dataset.groupby(["PATNR","DATUM"]).head(1)
dataset.DATUM =  dataset.DATUM.dt.date

1986


In [50]:
dataset[["TJC_28", "SJC_28"]].mean(), dataset[["TJC_28", "SJC_28"]].std()

(TJC_28    3.135850
 SJC_28    2.349905
 dtype: float64,
 TJC_28    4.187713
 SJC_28    3.470794
 dtype: float64)

In [51]:

dataset.to_csv(data_dir+"Leiden_mannequine_file.csv", index=False)


In [52]:
vas_data = vas_data.rename(columns={"DATUM_use":"DATUM"})
vas_data.query("`VAS Patient (mm)` > 100")

item_text,PATNR,DATUM,VAS Patient (mm),VAS arts (mm)


In [53]:
#dataset.to_csv(data_dir+"Leiden_mannequine_file.csv", index=False)

In [54]:
dataset.PATNR.nunique()

1986